In [2]:
from jupyter_dash import JupyterDash
from dash import html, dcc, Input, Output, dash_table  # pip install dash
import dash_bootstrap_components as dbc
import pandas as pd

import matplotlib.pyplot as plt             # pip install matplotlib
import mpld3                                # pip install mpld3


df = pd.read_csv("https://raw.githubusercontent.com/Coding-with-Adam/Dash-by-Plotly/master/Bootstrap/Berlin_crimes.csv")
print(df.head())


app = JupyterDash(__name__, external_stylesheets=[dbc.themes.BOOTSTRAP])
app.layout = dbc.Container([
    html.H1("Interactive Matplotlib with Dash", className='mb-2', style={'textAlign':'center'}),

    dbc.Row([
        dbc.Col([
            dcc.Dropdown(
                id='yeardropdown',
                value=2012,
                clearable=False,
                options=[{'label': x, 'value': x} for x in df.Year.unique()])
        ], width=6),
        dbc.Col([
            dcc.Dropdown(
                id='distdropdown',
                value='Mitte',
                clearable=False,
                options=[{'label': x, 'value': x} for x in
                         sorted(df.District.unique())])
        ], width=6),
    ]),
    dbc.Row([
        dbc.Col([
            html.Iframe(
                id='scatter-plot',
                srcDoc=None,  # here is where we will put the graph we make
                style={'border-width': '5', 'width': '100%',
                       'height': '400px'}),

        ], width=6),
        dbc.Col([
            html.Iframe(
                id='bar-plot',
                srcDoc=None,  # here is where we will put the graph we make
                style={'border-width': '5', 'width': '100%',
                       'height': '400px'}),

        ], width=6)
    ]),
    dbc.Row([
        dbc.Col([
            html.Div(id='table-placeholder', children=[])
        ], width=12),
    ]),

])

# Create interactivity between components and graph
@app.callback(
    Output('scatter-plot', 'srcDoc'),
    Output('bar-plot', 'srcDoc'),
    Output('table-placeholder', 'children'),
    Input('yeardropdown', 'value'),
    Input('distdropdown', 'value')
)
def plot_data(selected_year, selected_district):

    # filter data based on user selection
    dff = df[df.Year == selected_year]
    dff = dff[dff.District == selected_district]

    # build scatter plot
    fig, ax = plt.subplots()
    ax.scatter(x=dff.Damage, y=dff.Graffiti, s=dff.Drugs)
    ax.set_xlabel("Damage")
    ax.set_ylabel("Graffiti")
    ax.grid(color='lightgray', alpha=0.7)
    html_scatter = mpld3.fig_to_html(fig)

    # build bar plot
    fig, ax = plt.subplots()
    ax.bar(x=dff.Location, height=dff.Robbery)
    ax.set_xlabel(selected_district)
    ax.set_ylabel("Robbery")
    ax.grid(color='lightgray', alpha=0.7)
    html_bar = mpld3.fig_to_html(fig)

    # build DataTable
    mytable = dash_table.DataTable(
        id='table',
        columns=[{"name": i, "id": i} for i in dff.columns],
        data=dff.to_dict('records'),
    )

    return html_scatter, html_bar, mytable


app.run_server(mode='jupyterlab') # jupyterlab, inline

   Year District   Code           Location  Robbery  Street_robbery  Injury  \
0  2012    Mitte  10111     Tiergarten Süd       70              46     586   
1  2012    Mitte  10112  Regierungsviertel       65              29     474   
2  2012    Mitte  10113     Alexanderplatz      242             136    1541   
3  2012    Mitte  10114  Brunnenstraße Süd       52              25     254   
4  2012    Mitte  10221        Moabit West      130              51     629   

   Agg_assault  Threat  Theft  Car  From_car  Bike  Burglary  Fire  Arson  \
0          194     118   2263   18       328   120        68    16      4   
1          123     142   3203   10       307   170        37    10      4   
2          454     304   8988   81       792   822       275    49     27   
3           60      66   1916   86       192   396       131    14      5   
4          185     199   2470   94       410   325       161    42     22   

   Damage  Graffiti  Drugs  Local  
0     273        26    171

c:\users\13474\appdata\local\programs\python\python39\lib\site-packages\jupyter_dash\jupyter_app.py:139: UserWarning:

The 'environ['werkzeug.server.shutdown']' function is deprecated and will be removed in Werkzeug 2.1.

